In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/datasets/playground-series-s3e8/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/datasets/playground-series-s3e8/test.csv')

result = df_test['id']
result = pd.DataFrame(result)

In [3]:
df_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,193573,0.35,Ideal,D,VS2,62.3,56.0,4.51,4.54,2.82
1,193574,0.77,Very Good,F,SI2,62.8,56.0,5.83,5.87,3.68
2,193575,0.71,Ideal,I,VS2,61.9,53.0,5.77,5.74,3.55
3,193576,0.33,Ideal,G,VVS2,61.6,55.0,4.44,4.42,2.73
4,193577,1.20,Very Good,I,VS2,62.7,56.0,6.75,6.79,4.24


In [4]:
df_train.drop(['id'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [5]:
df_train.head()

,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.52,62.2,58.0,7.27,7.33,4.55,13619,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2.03,62.0,58.0,8.06,8.12,5.05,13387,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,0.70,61.2,57.0,5.69,5.73,3.50,2772,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,0.32,61.6,56.0,4.38,4.41,2.71,666,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,1.70,62.6,59.0,7.65,7.61,4.77,14453,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
df_test.head()

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.35,62.3,56.0,4.51,4.54,2.82,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0.77,62.8,56.0,5.83,5.87,3.68,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0.71,61.9,53.0,5.77,5.74,3.55,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,0.33,61.6,55.0,4.44,4.42,2.73,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,1.20,62.7,56.0,6.75,6.79,4.24,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [7]:
scaler = StandardScaler()
df_train[['carat', 'depth', 'table', 'x', 'y', 'z']] = scaler.fit_transform(df_train[['carat', 'depth', 'table', 'x', 'y', 'z']])
df_test[['carat', 'depth', 'table', 'x', 'y', 'z']] = scaler.fit_transform(df_test[['carat', 'depth', 'table', 'x', 'y', 'z']])

X = df_train.drop(['price'], axis=1)
y = df_train['price']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = df_test.copy()

y_train = np.log(y_train)
y_val = np.log(y_val)

In [9]:
num = [10, 20, 50]
RNG_SEED = 888
N_JOBS = 2
N_FOLDS = 5
SCORING_METRIC = 'neg_root_mean_squared_error'

In [10]:
train_models = []
train_results = []
train_model_names = []
train_metrics = []

In [11]:
train_models.append(('XGB', XGBRegressor(random_state=RNG_SEED, n_jobs=N_JOBS, objective='reg:squarederror', tree_method='gpu_hist')))

In [12]:
# Generate model in turn
for name, model in train_models:
    kfold = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RNG_SEED)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=SCORING_METRIC, n_jobs=2, verbose=1)
    train_results.append(cv_results)
    train_model_names.append(name)
    train_metrics.append(cv_results.mean())
    print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))
    print(model)
print ('Average metrics (' + SCORING_METRIC + ') from all models:', np.mean(train_metrics))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


XGB: -590.298461 (7.739975)
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=2, num_parallel_tree=None, predictor=None,
             random_state=888, ...)
Average metrics (neg_root_mean_squared_error) from all models: -590.2984607195618


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   11.2s finished


In [13]:
# Set up the comparison array
tune_results = []
tune_model_names = []

In [14]:
# Tuning XGBoost n_estimators, max_depth, and min_child_weight parameters
tune_model1 = XGBRegressor(random_state=RNG_SEED, n_jobs=N_JOBS, objective='reg:squarederror', tree_method='gpu_hist')
tune_model_names.append('XGB_1')
paramGrid1 = dict(n_estimators=range(100, 501, 100),
                  max_depth=np.array([3, 6, 9]),
                  min_child_weight=np.array([1, 2, 3]))

kfold = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RNG_SEED)
grid1 = GridSearchCV(estimator=tune_model1, param_grid=paramGrid1, scoring=SCORING_METRIC, cv=kfold, n_jobs=N_JOBS, verbose=1)
grid_result1 = grid1.fit(X, y)

print("Best: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
tune_results.append(grid_result1.cv_results_['mean_test_score'])
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 45 candidates, totalling 225 fits
Best: -589.572397 using {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 500}
-618.295311 (5.811178) with: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100}
-599.372735 (6.813373) with: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200}
-593.827432 (7.607274) with: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 300}
-591.641777 (7.557600) with: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 400}
-590.497366 (7.573350) with: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 500}
-617.883448 (5.227122) with: {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 100}
-600.367270 (6.105647) with: {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 200}
-592.637207 (7.095766) with: {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 300}
-590.580704 (7.244099) with: {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 400}
-589.572397 (7.715640) with: {'max_depth': 

In [15]:
BEST_MAX_DEPTH = grid_result1.best_params_['max_depth']
BEST_MIN_CHILD_WEIGHT = grid_result1.best_params_['min_child_weight']
BEST_N_ESTIMATORS = grid_result1.best_params_['n_estimators']
print('Best max_depth parameter:', BEST_MAX_DEPTH)
print('Best min_child_weight parameter:', BEST_MIN_CHILD_WEIGHT)
print('Best n_estimators parameter:', BEST_N_ESTIMATORS)

Best max_depth parameter: 3
Best min_child_weight parameter: 2
Best n_estimators parameter: 500


In [16]:
# Tuning XGBoost subsample and colsample_bytree parameters
tune_model2 = XGBRegressor(n_estimators=BEST_N_ESTIMATORS, max_depth=BEST_MAX_DEPTH, min_child_weight=BEST_MIN_CHILD_WEIGHT,
                            random_state=RNG_SEED, n_jobs=N_JOBS, objective='reg:squarederror', tree_method='gpu_hist')
tune_model_names.append('XGB_2')
paramGrid2 = dict(subsample=np.array([0.7, 0.8, 0.9, 1.0]),
                  colsample_bytree=np.array([0.7, 0.8, 0.9, 1.0]))

kfold = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RNG_SEED)
grid2 = GridSearchCV(estimator=tune_model2, param_grid=paramGrid2, scoring=SCORING_METRIC, cv=kfold, n_jobs=N_JOBS, verbose=1)
grid_result2 = grid2.fit(X, y)

print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
tune_results.append(grid_result2.cv_results_['mean_test_score'])
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best: -589.550486 using {'colsample_bytree': 0.7, 'subsample': 1.0}
-605.287205 (8.221407) with: {'colsample_bytree': 0.7, 'subsample': 0.7}
-601.038759 (7.957839) with: {'colsample_bytree': 0.7, 'subsample': 0.8}
-596.931068 (7.584260) with: {'colsample_bytree': 0.7, 'subsample': 0.9}
-589.550486 (6.771510) with: {'colsample_bytree': 0.7, 'subsample': 1.0}
-606.088531 (6.403926) with: {'colsample_bytree': 0.8, 'subsample': 0.7}
-601.871149 (8.314565) with: {'colsample_bytree': 0.8, 'subsample': 0.8}
-597.486358 (7.416392) with: {'colsample_bytree': 0.8, 'subsample': 0.9}
-589.618991 (7.753280) with: {'colsample_bytree': 0.8, 'subsample': 1.0}
-606.166962 (7.303183) with: {'colsample_bytree': 0.9, 'subsample': 0.7}
-601.454207 (7.893246) with: {'colsample_bytree': 0.9, 'subsample': 0.8}
-597.664765 (5.902669) with: {'colsample_bytree': 0.9, 'subsample': 0.9}
-590.049076 (7.051779) with: {'colsample_bytree': 0.9, 'subsample': 

In [17]:
BEST_COLSAMPLE_BYTREE = grid_result2.best_params_['colsample_bytree']
BEST_SUBSAMPLE = grid_result2.best_params_['subsample']
print('Best colsample_bytree parameter:', BEST_COLSAMPLE_BYTREE)
print('Best subsample parameter:', BEST_SUBSAMPLE)

Best colsample_bytree parameter: 0.7
Best subsample parameter: 1.0


In [19]:
final_model = XGBRegressor(n_estimators=BEST_N_ESTIMATORS, max_depth=BEST_MAX_DEPTH, min_child_weight=BEST_MIN_CHILD_WEIGHT,
                           colsample_bytree=BEST_COLSAMPLE_BYTREE, subsample=BEST_SUBSAMPLE, random_state=RNG_SEED,
                           n_jobs=N_JOBS, objective='reg:squarederror', tree_method='gpu_hist')
final_model.fit(X, y)
print(final_model)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=2, num_parallel_tree=None, predictor=None,
             random_state=888, ...)


In [20]:
test_predictions =  final_model.predict(X_test)
print(test_predictions)

[ 822.929  2413.54   2260.197  ... 7954.679  6302.2505 4047.2   ]


In [ ]:
test_predictions = np.exp(test_predictions)
test_predictions

In [21]:
result['price'] = test_predictions
result.to_csv('submission.csv', index=False)
result

,id,price
0,193573,822.929016
1,193574,2413.540039
2,193575,2260.197021
3,193576,886.542908
4,193577,5757.389160
...,...,...
129045,322618,4092.236572
129046,322619,2640.944580
129047,322620,7954.679199
129048,322621,6302.250488
